<a href="https://colab.research.google.com/github/badim04/Forex-Bot-Grid/blob/main/grid_system_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta



In [2]:
dataF = yf.download("GBPUSD=X", start="2024-05-19", end="2024-06-30", interval='30m')
#dataF.iloc[:,:]

[*********************100%%**********************]  1 of 1 completed


In [3]:
dataF.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-05-20 00:00:00,1.270132,1.270648,1.270132,1.270632,1.270632,0
2024-05-20 00:30:00,1.270648,1.270858,1.270632,1.270632,1.270632,0
2024-05-20 01:00:00,1.270664,1.270810,1.269809,1.270810,1.270810,0
2024-05-20 01:30:00,1.270729,1.270842,1.270438,1.270438,1.270438,0
2024-05-20 02:00:00,1.270325,1.270745,1.270309,1.270567,1.270567,0


In [4]:
#!pip install pandas_ta==0.3.14b

In [5]:
grid_distance = 0.008 # This means 80 pips
midprice = 1.2705 #

In [6]:
def generate_grid(midprice, grid_distance, grid_range):
    return (np.arange(midprice-grid_range, midprice+grid_range, grid_distance))

grid = generate_grid(midprice=midprice, grid_distance=grid_distance, grid_range=0.1)
grid


array([1.1705, 1.1785, 1.1865, 1.1945, 1.2025, 1.2105, 1.2185, 1.2265,
       1.2345, 1.2425, 1.2505, 1.2585, 1.2665, 1.2745, 1.2825, 1.2905,
       1.2985, 1.3065, 1.3145, 1.3225, 1.3305, 1.3385, 1.3465, 1.3545,
       1.3625, 1.3705])

In [7]:
signal = [0]*len(dataF)
i=0
for index, row in dataF.iterrows():
    for p in grid:
        if min(row.Low, row.High)<p and max(row.Low, row.High)>p:
            signal[i]=1
    i+=1

In [8]:
dataF["signal"]=signal
dataF[dataF["signal"]==1]

,Open,High,Low,Close,Adj Close,Volume,signal
Datetime,,,,,,,
2024-05-22 07:00:00,1.274535,1.275283,1.274129,1.274632,1.274632,0,1
2024-05-22 08:00:00,1.275836,1.275836,1.273561,1.273853,1.273853,0,1
2024-05-22 08:30:00,1.273804,1.275120,1.273545,1.274860,1.274860,0,1
2024-05-22 09:00:00,1.274778,1.274778,1.272653,1.272653,1.272653,0,1
2024-05-22 14:30:00,1.274324,1.274697,1.274080,1.274080,1.274080,0,1
...,...,...,...,...,...,...,...
2024-06-26 10:00:00,1.266672,1.266897,1.266368,1.266865,1.266865,0,1
2024-06-26 11:00:00,1.266817,1.266849,1.265679,1.265743,1.265743,0,1
2024-06-26 11:30:00,1.265775,1.266512,1.265759,1.265759,1.265759,0,1


In [9]:
dfpl = dataF[:].copy()
def SIGNAL():
    return dfpl.signal
dfpl['ATR'] = ta.atr(high = dfpl.High, low = dfpl.Low, close = dfpl.Close, length = 16)
dfpl.dropna(inplace=True)

In [10]:
#!pip install backtesting==0.2.1

In [11]:
#!pip install --upgrade backtesting

In [12]:
from backtesting import Strategy
from backtesting import Backtest
import backtesting

class MyStrat(Strategy):
    mysize = 50
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 1.5*grid_distance #5*self.data.ATR[-1]
        TPSLRatio = 0.5

        if self.signal1==1 and len(self.trades)<=10000:
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=50, margin=1/100, hedging=True, exclusive_orders=False)
stat = bt.run()
stat

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Start                     2024-05-20 08:00:00
End                       2024-06-28 22:00:00
Duration                     39 days 14:00:00
Exposure Time [%]                   93.243243
Equity Final [$]                    72.747839
Equity Peak [$]                     74.503017
Return [%]                          45.495678
Buy & Hold Return [%]               -0.357912
Return (Ann.) [%]                 2233.139828
Volatility (Ann.) [%]             1574.613265
Sharpe Ratio                         1.418215
Sortino Ratio                        73.39704
Calmar Ratio                        99.291477
Max. Drawdown [%]                   -22.49075
Avg. Drawdown [%]                    -1.38348
Max. Drawdown Duration        9 days 19:30:00
Avg. Drawdown Duration        0 days 15:40:00
# Trades                                  180
Win Rate [%]                        77.222222
Best Trade [%]                       0.690904
Worst Trade [%]                     -0.974186
Avg. Trade [%]                    

In [13]:
backtesting.set_bokeh_output(notebook=False)
bt.plot(show_legend=False, plot_width=None, plot_equity=True, plot_return=False,
plot_pl=False, plot_volume=False, plot_drawdown=False, smooth_equity=False, relative_equity=True,
superimpose=True, resample=False, reverse_indicators=False, open_browser=True)

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1252', ...)

In [14]:
stat._trades.sort_values(by="EntryBar").head(20)

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
1,-50,95,167,1.274600,1.268632,0.298379,0.004682,2024-05-22 07:30:00,2024-05-23 19:30:00,1 days 12:00:00
45,50,95,501,1.274600,1.280632,0.301621,0.004733,2024-05-22 07:30:00,2024-06-03 21:30:00,12 days 14:00:00
79,-50,97,819,1.273804,1.285853,-0.602432,-0.009459,2024-05-22 08:30:00,2024-06-12 14:00:00,21 days 05:30:00
9,50,97,297,1.273804,1.279853,0.302432,0.004748,2024-05-22 08:30:00,2024-05-28 14:00:00,6 days 05:30:00
0,-50,98,167,1.274778,1.268860,0.295935,0.004643,2024-05-22 09:00:00,2024-05-23 19:30:00,1 days 10:30:00
51,50,98,504,1.274778,1.280860,0.304065,0.004770,2024-05-22 09:00:00,2024-06-03 23:00:00,12 days 14:00:00
76,-50,99,818,1.272799,1.284653,-0.592710,-0.009313,2024-05-22 09:30:00,2024-06-12 13:30:00,21 days 04:00:00
6,50,99,293,1.272799,1.278653,0.292710,0.004599,2024-05-22 09:30:00,2024-05-28 12:00:00,6 days 02:30:00
25,50,110,499,1.274161,1.280080,0.295941,0.004645,2024-05-22 15:00:00,2024-06-03 20:30:00,12 days 05:30:00
108,-50,110,914,1.274161,1.268080,0.304059,0.004773,2024-05-22 15:00:00,2024-06-14 13:30:00,22 days 22:30:00
